In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
import copy
import random
import math
import time

import cma  # Importando a biblioteca CMA-ES

from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [16]:
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T

# Definindo os limites para os coeficientes
bounds = {
    'tau': (0.1, 10.0),
    'k': (0.1, 5.0),
    'n': (0.1, 5.0),
}

IND_SIZE = 19

class Coefficient:
    def __init__(self, bounds_val):
        self.val = random.uniform(*bounds_val)
        self.bounds_val = bounds_val
    
    def __repr__(self):
        return f"val={self.val}"
    
class Individual:
    def __init__(self):
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        self.ind_size = IND_SIZE
        self.fitness = np.inf
        # self.fitness = random.uniform(0, 1)
    
    def calc_fitness(self, method='RK45'):
        equation = Equation(self.numerical_coeffs, labels)
        y = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
        self.fitness = self.abs_error(original, y)
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
            
    @staticmethod
    def initialize_ind(bounds):
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = Coefficient(bounds['tau'])
            for sub_key, coeffs in label.items():
                if sub_key != 'tau':
                    coeffs['n'] = Coefficient(bounds['n'])
                    coeffs['k'] = Coefficient(bounds['k'])
        
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self):
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for sub_key, coeffs in label.items():
                if sub_key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)
                    coeffs['k'] = coeffs['k'].val
        return numerical_coeffs
    
    def plot(self, method='RK45'):
        methods = [method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual(fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"
           

In [19]:
bound = [
    (0.1, 10.0),  # Limites para 'tau'
    (0.1, 5.0),   # Limites para 'k'
    (0.1, 5.0),   # Limites para 'n'
]

# CMA-ES Optimization
def run_cma_es():
    # Definindo parâmetros de otimização
    options = {
        'bounds': bound,  # Usando os limites para o CMA-ES
        'maxiter': 100,  # Número máximo de iterações
        'popsize': 20,  # Tamanho da população
        'tolx': 1e-6,  # Tolerância para mudança nos parâmetros
    }
    
    # Função de fitness a ser minimizada
    def fitness_func(x):
        # Convertendo os parâmetros do CMA-ES para o formato adequado para o Individual
        coeffs_dict = {
            'A': {'tau': Coefficient((x[0], x[1])), 'E': {'n': Coefficient((x[2], x[3])), 'k': Coefficient((x[4], x[5])), '-': True}},
            'B': {'tau': Coefficient((x[6], x[7])), 'A': {'n': Coefficient((x[8], x[9])), 'k': Coefficient((x[10], x[11])), '-': False}},
            'C': {'tau': Coefficient((x[12], x[13])), 'B': {'n': Coefficient((x[14], x[15])), 'k': Coefficient((x[16], x[17])), '-': False}},
            'D': {'tau': Coefficient((x[18], x[19])), 'C': {'n': Coefficient((x[20], x[21])), 'k': Coefficient((x[22], x[23])), '-': False}},
            'E': {'tau': Coefficient((x[24], x[25])), 'D': {'n': Coefficient((x[26], x[27])), 'k': Coefficient((x[28], x[29])), '-': False},
                  'B': {'n': Coefficient((x[30], x[31])), 'k': Coefficient((x[32], x[33])), '-': False},
                  'E': {'n': Coefficient((x[34], x[35])), 'k': Coefficient((x[36], x[37])), '-': False}}
        }
        
        ind = Individual()
        ind.coeffs = coeffs_dict
        ind.calc_fitness()  # Calculando a fitness
        return ind.fitness

    # Inicializando o CMA-ES
    es = cma.CMAEvolutionStrategy(IND_SIZE * [0], 0.5, options)  # Indivíduo inicial e a largura da distribuição
    
    # Otimização com CMA-ES
    while not es.stop():
        
        # Gera a população de soluções
        solutions = es.ask()
        
        # Avalia a função objetivo para cada solução
        fitness_vals = [fitness_func(sol) for sol in solutions]
        
        # Fornece as avaliações de fitness ao CMA-ES
        es.tell(solutions, fitness_vals)
        
        es.result()
    
    return es.result()

# Rodando a otimização CMA-ES
result = run_cma_es()
print(f"Melhor solução encontrada: {result[0]}")

ValueError: bounds must be None, empty, or a list of length 2 where each element may be a scalar, list, array, or None; type(bounds) was: <class 'list'>